In [ ]:
%cd ..

In [ ]:
from time import sleep

import numpy as np
import matplotlib.pyplot as plt

from evgena.dataset import Dataset
from evgena.genetals import Images2LabelObjectiveFnc
from evgena.model import TfModel
from evgena.metrics import mse
from evgena.utils.large_files import maybe_download
from evgena.data_transformations import images_to_BHWC, augment_images
from genetals.core import *
from genetals.initializers import MultivariateRandomInit
from genetals.operators import ShuffleOperator, TwoPointXover, BiasedMutation, NSGAOperator
from genetals.callbacks import MultiObjectiveReport, GAStatus

# Images to label #
Demonstration of GA ability to generate adversarial pattern for misclassifying a set of images to given label.
- [experiment](#Experiment)
- [result inspection](#Result-inspection)

## Experiment ##

Model to be "attacked"

Set of images to convert

In [ ]:
model_path = 'models/2018-05-26_025632.fashion_mnist-b_64-lr_0002/30-best'

model = TfModel(model_path, batch_size=8192)
fashion_mnist = Dataset.from_nprecord(maybe_download('datasets/fashion_mnist.npz'))

source_class = 0
target_class = 5
source_dataset = Dataset.sub_dataset(fashion_mnist, [source_class], do_stratified=False)

target_images = source_dataset.train.X[0:1]

Visualizations

In [ ]:
%matplotlib ipympl

fig, ax = plt.subplots(1, 1, figsize=(9,4))
# fig.tight_layout()

ax.set_xlim(0.0000000001, 1)
ax.set_xlabel('Target class prediction probability')
ax.set_ylim(-0.05, 0)
ax.set_ylabel('similarity')
ax.set_xscale('log')
ax.grid(axis='both')
ax.vlines(0.5, -1, 1, colors='g')

GA definition

In [ ]:
initializer = MultivariateRandomInit.uniform(
    (28, 28),
    scales=np.random.permutation((np.exp(np.linspace(3, 5, 100)) - 1) / (np.exp(5) - 1))
)

graph = OperatorGraph()
select_op = ShuffleOperator(graph.init_op)
xover_op = TwoPointXover(select_op, 0.8)
mutation_op = BiasedMutation(xover_op, sigma=0.1, l_bound=-0.1, u_bound=0.1)
moea_op = NSGAOperator(graph.init_op, mutation_op)

callbacks = [MultiObjectiveReport(ax), GAStatus(fig)]

ga = GeneticAlgorithm(
    initializer=initializer,
    operator_graph=graph,
    objective_fnc=Images2LabelObjectiveFnc(
        model, lambda x, y: - mse(x, y),
        target_class, target_images,  # TODO fix one image
        sample_size=1, sample_ttl=1
    ),
    callbacks=callbacks,
    metadata={
        'dataset': target_images,
        'model': np.asarray(model_path),
        'source_class': np.asarray(source_class),
        'target_class': np.asarray(target_class)
    },
    results_dir='logs/images2label'
)

GA run

In [ ]:
%time final_pop, fitnesses, objectives = ga.run(population_size=512, generation_cap=256)

In [ ]:
%time final_pop, fitnesses, objectives = ga.resume(generation_cap=256)

## Result inspection ##

In [ ]:
log_path = 'logs/images2label/18-05-26-21-50-52.npz'

with np.load(log_path) as run_file:
    individuals = run_file['genes']
    objectives = run_file['objectives']
    fitnesses = run_file['fitnesses']
    dataset = run_file['dataset']
    model_path = run_file['model']
    source_class = run_file['source_class']
    target_class = run_file['target_class']
    
model = TfModel(str(model_path))

### Replay ###

In [ ]:
%matplotlib ipympl

fig, ax = plt.subplots(1, 1, figsize=(9,5))
# fig.tight_layout()

ax.set_xlim(0.0000000001, 1)
ax.set_xlabel('Target class prediction probability')
ax.set_ylabel('mean SSIM')
ax.set_xscale('log')
ax.grid(axis='both')
ax.vlines(0.5, -1, 1, colors='g')

In [ ]:
ax.set_xlim(objectives[..., 0].min(), 1)
ax.set_ylim(objectives[..., 1].min(), objectives[..., 1].max())

for epoch, curr_objectives in enumerate(objectives):
    ax.lines = []
    ax.plot(*curr_objectives.transpose(), 'r+')
    
    fig.canvas.set_window_title('Current generation: {}'.format(epoch))
    fig.canvas.draw()
    
    sleep(0.02)

### Vizualization ###

In [ ]:
prediction_bound = 0.5
sim_bound = -0.1

filtered_indices, *_ = np.where(np.logical_and(
    objectives[-1, :, 0] > prediction_bound,
    objectives[-1, :, 1] > sim_bound
))
filtered_individuals = individuals[filtered_indices]
filtered_objectives = objectives[-1, filtered_indices]

augmented_images = augment_images(filtered_individuals, dataset)
augmented_images_batch_shaped = augmented_images.reshape(-1, *augmented_images.shape[2:4], 1)

generalization = model(augmented_images_batch_shaped)[:, target_class].reshape(augmented_images.shape[:2])

print('Mean test prediction: {:4f}, target predicted {} out of {}.'.format(generalization.mean(), np.sum(generalization > 0.5), generalization.size))

leaderboard = np.flip(np.argsort(filtered_objectives[:,1]), 0)

In [ ]:
%matplotlib ipympl

fig, ax = plt.subplots(8, 8, figsize=(8, 8))

for i in range(64):
    if i > len(filtered_individuals):
        break
    
    ax[i // 8, i % 8].axis('off')
    ax[i // 8, i % 8].imshow(filtered_individuals[i], cmap='plasma', vmin=-0.2, vmax=0.2)

In [ ]:
%matplotlib ipympl

compare_fig, compare_ax = plt.subplots(1, 3, figsize=(9, 3))
compare_fig.tight_layout()

In [ ]:
individual_i = leaderboard[0]
image_i = 0

compare_ax[0].imshow(dataset[image_i], cmap='gray', vmin=0, vmax=1)
compare_ax[1].imshow(filtered_individuals[individual_i], cmap='plasma', vmin=-1, vmax=1)
compare_ax[2].imshow(augmented_images[individual_i, image_i][:,:,0], cmap='gray', vmin=0, vmax=1)

compare_fig.canvas.set_window_title('Target score: {}'.format(
    model(augmented_images[individual_i, image_i])[0, target_class]
))
compare_fig.canvas.draw()

### TODO ###
- make some nice GUI, widgets etc., notebook/lab compatibility?